In [1]:
#!pip install torch transformers nltk scikit-learn datasets
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split   
import os
#blogs_raw = load_dataset('blog_authorship_corpus',cache_dir='/home2/cgmj52/ResearchProject/Data/Datasets' )

scientific_raw = load_dataset('scientific_papers', 'arxiv', trust_remote_code=True,cache_dir='/home2/cgmj52/ResearchProject/Data/Datasets' )
journalistic_raw = load_dataset('multi_news', trust_remote_code=True , )
stories_raw = load_dataset('roneneldan/tinystories', )
gpt_dataset = load_dataset("aadityaubhat/GPT-wiki-intro",cache_dir='/home2/cgmj52/ResearchProject/Data/Datasets')
#gpt2_data = load_dataset("spacerini/gpt2-outputs", cache_dir='/home2/cgmj52/ResearchProject/Data/Datasets' )


import re




Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'wiki_intro', 'generated_intro', 'title_len', 'wiki_intro_len', 'generated_intro_len', 'prompt', 'generated_text', 'prompt_tokens', 'generated_text_tokens'],
        num_rows: 150000
    })
})
DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6440
    })
})
DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})
DatasetDict({
    train: Dataset({
        features: ['text'],
        

In [2]:
from nltk import word_tokenize

def preprocess_data(text, sentence_length=20):
    # Remove LaTeX commands
    text = re.sub(r'\\[a-zA-Z]+', '', text)
    
    # Remove specific LaTeX artifacts like @math1 and \cite{...}
    text = re.sub(r'@math\d+', '', text)  # Assuming @math1, @math2, etc.
    text = re.sub(r'@xmath\d+', '', text) 
    text = re.sub(r'\\cite\{[^}]*\}', '', text)  # Remove \cite{...}
    text = re.sub(r'&nbsp;','', text)
    # Normalize ellipses and multiple exclamation/question marks to a single instance
    text = re.sub(r'\.\.\.+', '…', text)
    text = re.sub(r'!{2,}', '!', text)
    text = re.sub(r'\?{2,}', '?', text)
    text = re.sub(('([^\s\w]|_)+'), ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    final_array = []
    line = []
    for word in word_tokenize(text):
        # If adding the next word exceeds the sentence_length, write the current line and start a new one
        if len(line) >= sentence_length :
            final_array.append(' '.join(line))
            line = [word]  # Start a new line with the current word
        else:
            line.append(word)
        
            # Write any remaining words in the last line
    #final_array.append(' '.join(line))

    return final_array

def write_text_to_file(dataset, output_file, sentence_length, max_lines=500000):
    line_count = 0
    with open(output_file, 'w', encoding='utf-8') as f:
        for example in dataset:
            for line in example:
                if line_count > max_lines:
                    print(line_count)
                    return line_count
                line_count += 1
                f.write(line + '\n')
           
                
        print(line_count)
        return line_count

In [3]:
sentence_length = 20
gpt = list(map(preprocess_data, gpt_dataset['train']['generated_intro']))

train, test, = train_test_split(gpt, test_size=0.1, random_state=42)
test, dev = train_test_split(test, test_size=0.5, random_state=42)

max_lines = write_text_to_file(train, 'gpt.train', sentence_length )
test_lines = write_text_to_file(test, 'gpt.test', sentence_length )
dev_lines = write_text_to_file(dev, 'gpt.dev', sentence_length )
write_text_to_file(gpt, 'gpt.whole', sentence_length )
#gpt2_data = map(preprocess_text, gpt2_data['train']['text'])


500001
46032
45657
500001


500001

In [ ]:
scientific = list(map(preprocess_data,scientific_raw['train']['abstract']))
train, test, = train_test_split(scientific, test_size=0.3, random_state=42)
test, dev = train_test_split(test, test_size=0.5, random_state=42)

write_text_to_file(train, 'scientific.train', sentence_length, max_lines-1 )
write_text_to_file(test, 'scientific.test', sentence_length, test_lines-1 )
write_text_to_file(dev, 'scientific.dev', sentence_length, dev_lines-1 )

write_text_to_file(scientific, 'scientific.whole', sentence_length )
write_text_to_file(scientific + gpt, 'gpt_sci.corpus', sentence_length, 1000000)

In [15]:
write_text_to_file(scientific + gpt, 'gpt_sci.corpus', sentence_length, 1000000)

1000001


1000001

In [ ]:
journals_train = journalistic_raw['train']['abstract'].map(preprocess_data)
journals_test = journalistic_raw['test']['abstract'].map(preprocess_data)
journals_val = journalistic_raw['validation']['abstract'].map(preprocess_data)
write_text_to_file(journals_train, 'journals.train', sentence_length )
write_text_to_file(journals_test, 'journals.test', sentence_length )
write_text_to_file(journals_val, 'journals.dev', sentence_length )

In [ ]:
narrative = narrative_raw['train']['text'].map(preprocess_data)
train, test, = train_test_split(narrative, test_size=0.3, random_state=42)
test, dev = train_test_split(test, test_size=0.5, random_state=42)
write_text_to_file(train, 'narrative.train', sentence_length )
write_text_to_file(test, 'narrative.test', sentence_length )
write_text_to_file(dev, 'narrative.dev', sentence_length )

### Data Analysis

In [11]:
#generate samples
gpt_sample = gpt_dataset['train']['generated_intro'][:500]
stories_sample = stories_raw['train']['text'][:500]
journal_sample = journalistic_raw['train']['document'][:500]
scientific_sample = scientific_raw['train']['abstract'][:500]

#create an itterable so we can process each dataset
datasets = [gpt_sample, stories_sample, journal_sample, scientific_sample]
#https://www.geeksforgeeks.org/how-to-generate-word-embedding-using-bert/
# importing libraries
import random
import torch
from transformers import BertTokenizer, BertModel


# Set a random seed
random_seed = 42
random.seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    device = torch.device('cuda')

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
word_embedding_datasets = []
embeddings_labels = []
labels = ['GPT style', 'stories', 'news reports', 'scientific']

In [13]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
for y, dataset in enumerate(datasets):
    encoding = tokenizer.batch_encode_plus(
        dataset,                    # List of input texts
        padding=True,              # Pad to the maximum sequence length
        truncation=True,           # Truncate to the maximum sequence length if necessary
        return_tensors='pt',      # Return PyTorch tensors
        add_special_tokens=True    # Add special tokens CLS and SEP
    )
 
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    # Generate embeddings using BERT model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state[:,-1,:].numpy()  # This contains the embeddings
        word_embedding_datasets.append(word_embeddings)
        embeddings_labels += [y]*len(word_embeddings)
    print(f'completed dataset....{labels[y]}')

#max_length = max(len(seq) for seq in word_embedding_datasets)
#word_embedding_datasets = [np.pad(seq, ((0, max_length - len(seq)), (0, 0)), mode='constant') for seq in word_embedding_datasets]
word_embedding_datasets = np.concatenate(word_embedding_datasets, axis=0)
print(word_embedding_datasets.shape)

(1536000,)


In [ ]:

#due to the number of data points and dimensionality. It is best to use the GPU
#to install tsne cuda use: (must have python 3.9)
##cuda version is 12.4
'''!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.4.* dask-cudf-cu12==24.4.* cuml-cu12==24.4.* \
    cugraph-cu12==24.4.* cuspatial-cu12==24.4.* cuproj-cu12==24.4.* \
    cuxfilter-cu12==24.4.* cucim-cu12==24.4.* pylibraft-cu12==24.4.* \
    raft-dask-cu12==24.4.* cuvs-cu12==24.4.*'''


perplexity = np.arange(10,500, 50)
divergence = []

for i in perplexity:
    model = TSNE(n_components=2, perplexity=i)
    reduced = model.fit_transform(all_embeddings)
    divergence.append(model.kl_divergence_)

fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

#takes around 4 hrs

In [ ]:
# Plot the t-SNE results
model = TSNE(n_components=2, perplexity=500)
tsne_embeddings = model.fit_transform(all_embeddings)

plt.figure(figsize=(8, 6))
colors = ['red', 'blue', 'green', 'orange']
for i, label in enumerate(labels):
    mask = [x == i for x in embeddings_labels]
    plt.scatter(tsne_embeddings[mask, 0], tsne_embeddings[mask, 1], c=colors[i], alpha=0.8, label=label)

plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title('t-SNE Plot of Word Embeddings')
plt.legend()
plt.show()

### N-grams and callocations

In [ ]:
import nltk
from nltk import word_tokenize, ngrams
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
flattened_datasets = []
for dataset in datasets:
    #flatten the preprocessed lists
    flattened_dataset = []
    for i in range(len(dataset)):
        final_sentence =[]
        for sentence in dataset[i]:
            final_sentence += sentence 
        flattened_dataset.append(final_sentence)
    flattened_datasets.append(flattenend_dataset)

for i, dataset in enumerate(flattened_datasets):
    tokenized_sentences = [word_tokenize(sentence) for sentence in dataset]
    sentence_lengths = [len(sentence) for sentence in tokenized_sentences]
    #tokenize and get the sentence lengths
    avg_sentence_length = sum(sentence_lengths) / len(sentence_lengths)
    print(f"Average sentence length for {labels[i]}: {avg_sentence_length:.2f} words")
    # Flatten the tokenized sentences into a single list of words
    words = [word for sentence in tokenized_sentences for word in sentence]

    # Create a BigramCollocationFinder object
    finder = BigramCollocationFinder.from_words(words)

    # Apply frequency filter to remove rare collocations
    finder.apply_freq_filter(3)  # Adjust the frequency threshold as needed

    # Extract the top collocations
    top_collocations = finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)  # Adjust the number of top collocations as needed

    print(f"Top Collocations for {label[i]}")
    for collocation in top_collocations:
        print(collocation)
    print('\n\n')
    
    # Specify the value of N for N-grams
    n = 3  # Change this to the desired value of N

    # Extract N-grams from the tokenized sentences
    n_grams = [list(ngrams(sentence, n)) for sentence in tokenized_sentences]

    # Flatten the list of N-grams
    flat_n_grams = [gram for sublist in n_grams for gram in sublist]

    # Count the frequency of each N-gram
    n_gram_freq = nltk.FreqDist(flat_n_grams)

    # Print the most common N-grams
    print(f"Most common {n}-grams for {labels[i]}:")
    for n_gram, freq in n_gram_freq.most_common(10):  # Adjust the number of most common N-grams as needed
        print(f"{n_gram}: {freq}")